In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import deepmatcher as dm
import random as rd
from utils.dataset_parser import generate_unlabeled
from similarity.jaccard import Jaccard
from similarity.shingle_based import ShingleBased

In [14]:
comparator = Jaccard(3)
comparator.similarity('ciao belli mucconi','ciao biddi mucconi')

0.47368421052631576

In [7]:
tableA = pd.read_csv('Structured/itunes-amazon/exp1/tableA.csv')
tableB = pd.read_csv('Structured/itunes-amazon/exp1/tableB.csv')

In [19]:
def filter_fn(row,comparator,att,val,threshold):
    if comparator.similarity(row[att],val) >=threshold:
        return False
    else:
        return True

In [35]:

from tqdm import tqdm


##the attributes are in order of importance
def createPossibleMatchings(df1,df2,important_attributes,min_similarity,newsample_len):
    comparator = Jaccard(3)
    df1_index = list(df1.index)
    df2_index = list(df2.index)
    newSamples = []
    while(len(newSamples)<newsample_len):
        current_lrow = df1.iloc[rd.choice(df1.index)]
        current_df = df2
        for att in important_attributes:
            l_att = current_lrow[att]
            mask = current_df.apply(lambda row: comparator.similarity(row[att],l_att)>=min_similarity,axis=1)
            current_df = current_df[mask]
        for idx in current_df.index:
            newSamples.append((current_lrow['id'],current_df.at[idx,'id']))
    return newSamples

In [44]:
newSamples = createPossibleMatchings(tableA,tableB,['Artist_Name','Album_Name','Song_Name'],0.3,50)

In [48]:
newSamples_ids = pd.DataFrame(data=newSamples,columns=['ltable_id','rtable_id'])
newSamples_ids.to_csv('Structured/itunes-amazon/exp1/unlabeled_ids.csv',index=False)

In [49]:
newSamples_df = generate_unlabeled('Structured/itunes-amazon/exp1/','unlabeled_ids.csv')
newSamples_df.shape

(50, 17)

In [51]:
newSamples_df.tail(10)

,ltable_Song_Name,ltable_Artist_Name,ltable_Album_Name,ltable_Genre,ltable_Price,ltable_CopyRight,ltable_Time,ltable_Released,rtable_Song_Name,rtable_Artist_Name,rtable_Album_Name,rtable_Genre,rtable_Price,rtable_CopyRight,rtable_Time,rtable_Released,id
40,Call Me,Shinedown,The Sound of Madness,"Rock , Music , Metal , Alternative , Grunge , ...",$ 1.29,2008 Atlantic Recording Corp. for the United S...,3:42,24-Jun-08,Call Me,Shinedown,The Sound Of Madness,Rock,$ 1.29,2005 Z Records,3:42,"June 24 , 2008",40
41,Call Me,Shinedown,The Sound of Madness,"Rock , Music , Metal , Alternative , Grunge , ...",$ 1.29,2008 Atlantic Recording Corp. for the United S...,3:42,24-Jun-08,Call Me,Shinedown,The Sound of Madness [ Deluxe ],Rock,$ 1.29,( C ) 2006 Vanguard,3:42,"June 23 , 2008",41
42,Fly from the Inside ( Live Acoustic from WXDX ...,Shinedown,Leave a Whisper ( Deluxe Version ),"Rock , Music , Hard Rock , Alternative",$ 1.29,2009 Atlantic Recording Corporation for the Un...,3:59,27-May-03,Fly From The Inside [ Live Acoustic From Wxdx ...,Shinedown,Leave A Whisper ( Deluxe ),Rock,$ 1.29,2015 Mad Decent,3:59,"May 27 , 2003",42
43,Brother,X Ambassadors,Love Songs Drug Songs - EP,"Alternative , Music , Rock , Adult Alternative",$ 1.29,2013 KIDinaKORNER/Interscope Records,3:48,7-May-13,Brother,X Ambassadors,Love Songs Drug Songs,Alternative Rock,$ 1.29,( C ) 2013 KIDinaKORNER/Interscope Records,3:47,"May 7 , 2013",43
44,Brother,X Ambassadors,Love Songs Drug Songs - EP,"Alternative , Music , Rock , Adult Alternative",$ 1.29,2013 KIDinaKORNER/Interscope Records,3:48,7-May-13,Brother,X Ambassadors,Love Songs Drug Songs,Alternative Rock,$ 1.29,( C ) 2013 KIDinaKORNER/Interscope Records,3:47,"May 7 , 2013",44
45,It Goes Like This,Thomas Rhett,It Goes Like This,"Country , Music , Contemporary Country , Honky...",$ 1.29,"‰ ãÑ 2013 The Valory Music Co. , LLC",3:07,29-Oct-13,It Goes Like This,Thomas Rhett,It Goes Like This,Country,$ 1.29,"( C ) 2013 The Valory Music Co. , LLC",3:06,"October 29 , 2013",45
46,It Goes Like This,Thomas Rhett,It Goes Like This,"Country , Music , Contemporary Country , Honky...",$ 1.29,"‰ ãÑ 2013 The Valory Music Co. , LLC",3:07,29-Oct-13,It Goes Like This,Thomas Rhett,It Goes Like This,Country,$ 1.29,"( C ) 2013 The Valory Music Co. , LLC",3:06,"October 29 , 2013",46
47,It Goes Like This,Thomas Rhett,It Goes Like This,"Country , Music , Contemporary Country , Honky...",$ 1.29,"‰ ãÑ 2013 The Valory Music Co. , LLC",3:07,29-Oct-13,It Goes Like This,Thomas Rhett,It Goes Like This,Country,$ 1.29,"( C ) 2013 The Valory Music Co. , LLC",3:06,"October 29 , 2013",47
48,It Goes Like This,Thomas Rhett,It Goes Like This,"Country , Music , Contemporary Country , Honky...",$ 1.29,"‰ ãÑ 2013 The Valory Music Co. , LLC",3:07,29-Oct-13,It Goes Like This,Thomas Rhett,It Goes Like This,Country,$ 1.29,"( C ) 2013 The Valory Music Co. , LLC",3:06,"October 29 , 2013",48
49,Goodbye to You ( feat . Dot Rotten ),Ed Sheeran,5,"Pop , Music , Rock , Singer/Songwriter , Conte...",$ 1.29,2014 Warner Music UK LImited,5:30,23-Jun-14,Goodbye To You ( + Dot Rotten ),Ed Sheeran,5,Pop,$ 1.29,Dub Police Records,5:30,"June 23 , 2014",49


In [31]:
newSamples_df.to_csv('itunes_amazon_unlabeled.csv',index=False)

In [32]:
model = dm.MatchingModel(attr_summarizer='hybrid')
model.load_state('models/itunes_amazon_hybrid.pth')

In [34]:
unlabeled_dm = dm.data.process_unlabeled('itunes_amazon_unlabeled.csv',model,ignore_columns=['id'])
predictions = model.run_prediction(unlabeled_dm,output_attributes=True)


Reading and processing data from "itunes_amazon_unlabeled.csv"
0% [############################# ] 100% | ETA: 00:00:00

===>  PREDICT Epoch 6
Finished Epoch 6 || Run Time:    0.4 | Load Time:    0.2 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s:   0.00



In [38]:
matching_predictions = predictions[predictions.match_score <=0.5]
matching_predictions.head()

,match_score,ltable_Song_Name,ltable_Artist_Name,ltable_Album_Name,ltable_Genre,ltable_Price,ltable_CopyRight,ltable_Time,ltable_Released,rtable_Song_Name,rtable_Artist_Name,rtable_Album_Name,rtable_Genre,rtable_Price,rtable_CopyRight,rtable_Time,rtable_Released
id,,,,,,,,,,,,,,,,,
3,0.358066,Hourglass ( feat . Jessica Sanchez ) [ iTunes ...,Zedd,iTunes Session - EP,"Dance , Music",$ 1.29,2013 Interscope Records,3:42,19-Nov-13,Look At Me Now ( Clean Version ) [ Clean ],Alyxx Dione feat . Jason Derulo,Chris Brown and More,Pop,$ 1.29,( C ) 2008 Chris Young,3:42,NaN
5,0.162936,Thriller ( Live From Hammersmith Palais ),Fall Out Boy,Infinity On High - Deluxe Bonus - EP,"Alternative , Music , Punk , Rock , Adult Alte...",$ 0.99,2007 The Island Def Jam Music Group,3:32,20-Nov-07,Adrenaline [ Clean ],Blake Shelton,Bass Drops & Fist-Pumps,Country,$ 0.89,( c ) 2004 Los Cangris Music,3:32,NaN
9,0.357007,The Edge of Glory ( Live from `` A Very Gaga T...,Lady Gaga,A Very Gaga Holiday ( Live ) - EP,"Pop , Music , Dance , Rock , Pop/Rock , R&B / ...",$ 1.29,2011 Interscope Records,4:52,22-Nov-11,N.Y. State Of Mind ( Explicit Album Version ),Calvin Harris feat . HAIM,Pre-game Throwback,Dance & Electronic,$ 0.99,"( C ) 2015 Third Pardee Records , LLC under ex...",4:52,NaN
13,0.159220,Good To Be Alive ( Hallelujah ),Andy Grammer,Good To Be Alive ( Hallelujah ) - Single,"Pop , Music",$ 1.29,2015 S-Curve Records,3:09,24-Aug-15,Wild Child ( [ with Grace Potter ] ),Kenny Chesney,The Big Revival,Country,$ 1.29,2008 dirtybird,3:09,"September 22 , 2014"
23,0.375854,"You 're No Good ( feat . Santigold , Vybz Kart...",Major Lazer,Free the Universe ( Extended Version ),"Electronic , Music , Rock , Reggae , Modern Da...",$ 1.29,‰ ãÑ 2013 Secretly Canadian / Mad Decent,4:03,18-Jun-13,Betrayal,Brian Tyler,Thor : The Dark World ( Original Motion Pictur...,Soundtracks,$ 1.29,( C ) 2013 MARVEL,4:03,"November 5 , 2013"


In [ ]:
newSamples[0]

In [14]:

print(jarowink.similarity('Ciao belli','Ciao biddi'))

0.9199999999999999


In [44]:
newSamples_df.iloc[0]

ltable_Song_Name                 Home Sweet Home ( feat . Vince Neil )
ltable_Artist_Name                                        Justin Moore
ltable_Album_Name     Nashville Outlaws : A Tribute to MÌ ¦ tley CrÌ_e
ltable_Genre                                           Country , Music
ltable_Price                                                    $ 1.29
ltable_CopyRight                 ‰ ãÑ 2014 Big Machine Records , LLC .
ltable_Time                                                       3:50
ltable_Released                                              19-Aug-14
rtable_Song_Name                                    Say ( All I Need )
rtable_Artist_Name                                         OneRepublic
rtable_Album_Name                                    Dreaming Out Loud
rtable_Genre                                                       Pop
rtable_Price                                                    $ 1.29
rtable_CopyRight            ( C ) 2007 Mosley Music/Interscope Records
rtable